# 挑选长期成长的价值股

In [1]:
import pandas as pd
import numpy as np
import time

import datetime

import sys
sys.path.append("../utils/")
import token_util
pro = token_util.set_token()

*获取股票列表*

In [2]:
stock_list = pd.read_csv("../data_pulled/stock_date_delta_price(10, 200).csv")
stock_list.head()

,ts_code,name,total_mv,pe_ttm,price
0,000021.SZ,深科技,2.740956e+06,77.8015,18.840
1,000025.SZ,特力A,7.504725e+05,36.2790,17.485
2,000028.SZ,国药一致,1.743333e+06,14.2903,40.705
3,000032.SZ,深桑达A,7.355310e+05,59.1230,17.985
4,000034.SZ,神州数码,1.498475e+06,21.3637,23.190


In [3]:
stock_list.shape

(549, 5)

### 价值股

1.3年平均ROE>15%，最低的年份>10%</br>
2.3年内现金与约当现金占资产比率最低>10%，3年平均大于15%</br>
3.3年内净利润增长率未出现负增长，3年平均大于15%</br>
4.3年平均流动比率>150%，3年平均速动比率>100%</br>
5.3年平均毛利率>20%,3年平均净利率>10%</br>
6.净利润现金含量>70%

*获取数据*

In [4]:
# 获取 roe
def get_roe(ts_code, time_list):
    for i in range(len(time_list)):
                
        ret = pro.fina_indicator(ts_code=ts_code, period='20'+str(time_list[i])+'1231', \
                                 fields='ts_code,end_date,roe')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret
    
    return rets.drop_duplicates()[:len(time_list)].reset_index(drop=True)
    

# 获取 income 、毛利和净利润
def get_income(ts_code, time_list):
    for i in range(len(time_list)):

        ret = pro.income(ts_code=ts_code, period='20'+str(time_list[i])+'1231', \
                         fields='ts_code,end_date,n_income,revenue,oper_cost')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret
        
    rets['mao_li'] = (rets['revenue']-rets['oper_cost'])/rets['revenue']
    rets['jing_li'] = rets['n_income']/rets['revenue']

    return rets.drop_duplicates()[:len(time_list)].reset_index(drop=True)

# 获取资产负债信息
def get_balancesheet(ts_code, time_list):
    for i in range(len(time_list)):
        
        ret = pro.balancesheet(ts_code=ts_code, period='20'+str(time_list[i])+'1231', \
                               fields='ts_code, end_date, inventories, total_cur_assets, \
                               total_cur_liab, money_cap, total_assets')
        rets = pd.concat([rets, ret], ignore_index=True) if i != 0 else ret
        
    rets["liquidity_ratio"] = rets["total_cur_assets"]/rets["total_cur_liab"] * 100
    rets["quick_ratio"] = (rets["total_cur_assets"]-rets["inventories"])/rets["total_cur_liab"] * 100
    rets["cash_ratio"] = rets["money_cap"]/rets["total_assets"] * 100
            
    rets = rets.drop(labels=["inventories","total_cur_assets","total_cur_liab","money_cap","total_assets"], axis=1)
    
    return rets.drop_duplicates()[:len(time_list)].reset_index(drop=True)

*获取处理后的数据*

In [5]:
def get_data(stock_code, time_list):
    data = get_roe(stock_code, time_list)
    ret1 = get_income(stock_code, time_list)
    ret2 = get_balancesheet(stock_code, time_list)

    data["n_income"] = ret1["n_income"]
    data["mao_li"] = ret1["mao_li"]
    data["jing_li"] = ret1["jing_li"]
    data["liquidity_ratio"] = ret2["liquidity_ratio"]
    data["quick_ratio"] = ret2["quick_ratio"]
    data["cash_ratio"] = ret2["cash_ratio"]
    
    return data

*进行筛选*

In [6]:
candidate_list = []
time_list = [16, 17, 18, 19]
times = 0
starttime = datetime.datetime.now()
len_stock = len(stock_list)

while True:
    
    # 发生中断后，防止遗漏
    start_index = 0
    
    for cur_index in range(start_index, len_stock):
        
        start_index = cur_index

        try:
            data = get_data(stock_list["ts_code"][cur_index], time_list)
        except:
            time.sleep(5)
        
        times += 1
        
        if len(data) == 4:
            n_income_last = np.array(data.iloc[:len(data)-1]["n_income"])
            n_income_cur = np.array(data.iloc[1:]["n_income"])
            net_profit_growth_rate = ((n_income_cur - n_income_last) / n_income_last) * 100
            growth_rate_min = net_profit_growth_rate.min()
            growth_rate_mean = net_profit_growth_rate.mean()

            if data["roe"][1:].min() > 10 and data["roe"][1:].mean() > 15:
                 if data["cash_ratio"][1:].min() > 10 and data["cash_ratio"][1:].mean() > 15:
                    if growth_rate_min > 0 and growth_rate_mean > 15:
                        if data["liquidity_ratio"][1:].mean() > 150 and data["quick_ratio"][1:].mean() > 100:
                            candidate_list.append(stock_list["ts_code"][cur_index]) 

        if cur_index != 0 and cur_index % 100 == 0:
            endtime = datetime.datetime.now()
            print("task accomplish: {} %, cost time : {} minutes".format(round((cur_index+1)/len_stock, 2), \
                                            round((endtime - starttime).seconds / 60, 2)))

    if cur_index >= len_stock - 1:
        endtime = datetime.datetime.now()
        print("finish task !! total time : {} minutes".format(round((endtime - starttime).seconds / 60, 2)))
        break

task accomplish: 0.18 %, cost time : 2.55 minutes
task accomplish: 0.37 %, cost time : 5.47 minutes
task accomplish: 0.55 %, cost time : 8.52 minutes
task accomplish: 0.73 %, cost time : 11.3 minutes
task accomplish: 0.91 %, cost time : 14.05 minutes
finish task !! total time : 15.23 minutes


In [7]:
len(candidate_list)

55

In [8]:
candidate_list

['000568.SZ',
 '000596.SZ',
 '000858.SZ',
 '002035.SZ',
 '002127.SZ',
 '002262.SZ',
 '002271.SZ',
 '002396.SZ',
 '002402.SZ',
 '002415.SZ',
 '002439.SZ',
 '002572.SZ',
 '002677.SZ',
 '002690.SZ',
 '002737.SZ',
 '002777.SZ',
 '002821.SZ',
 '002832.SZ',
 '002867.SZ',
 '002901.SZ',
 '002912.SZ',
 '002918.SZ',
 '002947.SZ',
 '600009.SH',
 '600167.SH',
 '600197.SH',
 '600276.SH',
 '600406.SH',
 '600436.SH',
 '600486.SH',
 '600511.SH',
 '600763.SH',
 '600809.SH',
 '601100.SH',
 '601360.SH',
 '601888.SH',
 '603039.SH',
 '603060.SH',
 '603208.SH',
 '603283.SH',
 '603288.SH',
 '603338.SH',
 '603369.SH',
 '603393.SH',
 '603517.SH',
 '603568.SH',
 '603583.SH',
 '603589.SH',
 '603605.SH',
 '603659.SH',
 '603666.SH',
 '603801.SH',
 '603866.SH',
 '603882.SH',
 '603960.SH']

In [9]:
stock_list[(stock_list["ts_code"] == stock_list["ts_code"][cur_index])]

,ts_code,name,total_mv,pe_ttm,price
548,603990.SH,麦迪科技,544168.6611,157.4574,48.6
